### Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.extend(['/home/nkinsky/Documents/GitHub/NeuroPy'])
sys.path.extend(['/home/nkinsky/Documents/GitHubPrivate/pythonprogs/DataPaths/'])
    
import matplotlib.pyplot as plt
import numpy as np
import subjects
import pandas as pd
import seaborn as sns
from neuropy.plotting import Fig
from scipy import stats
from plotters import Plotter, get_nsd_vs_sd_df
from stats_utils import get_bootstrap_prob, get_bootstrap_prob_paired

grpdata = subjects.GroupData()

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy import plotting
import subjects

### An example 3d scatter plot showing sleep scoring parameters

In [2]:
from neuropy.analyses.brainstates import correlation_emg
from neuropy.utils.signal_process import FourierSg
from scipy.ndimage import gaussian_filter1d

smooth = lambda arr: gaussian_filter1d(arr, sigma=2 / 1)

sess = subjects.nsd.ratSday2[0]
maze = sess.paradigm["maze"].flatten()
post = sess.paradigm["post"].flatten()
period = [post[0], post[0] + 2 * 3600]

all_channels = np.concatenate(sess.probegroup.get_connected_channels()).astype("int")
signal = sess.eegfile.get_signal(t_start=period[0], t_stop=period[1])
emg = correlation_emg(signal, sess.probegroup, 10, 9)
emg = smooth(emg)

freqs = np.geomspace(0.5, 100, 100)
spect_kw = dict(window=10, overlap=9, freqs=freqs, norm_sig=True)
delta_lfp = sess.eegfile.get_signal(9, *period)
theta_lfp = sess.eegfile.get_signal(13, *period)
sg_deltachan = FourierSg(delta_lfp, **spect_kw)
delta = smooth(sg_deltachan.delta)
sg_thetachan = FourierSg(theta_lfp, **spect_kw)
theta = smooth(sg_thetachan.theta)

delta_full = smooth(sg_thetachan.get_band_power(2, 20))
theta_ratio = theta / delta_full

ImportError: cannot import name 'correlation_emg' from 'neuropy.analyses.brainstates' (/home/nkinsky/Documents/GitHub/NeuroPy/neuropy/analyses/brainstates.py)

In [ ]:
states = sess.brainstates.time_slice(*period)
color = np.array(["b"] * len(emg), dtype="U1")

time = sg_thetachan.time[:-1]

for e in states.itertuples():
    if e.label == "AW":
        col = "k"
    if e.label == "QW":
        col = "k"
    if e.label == "REM":
        col = "g"
    if e.label == "NREM":
        col = "b"

    color[(time > e.start) & (time < e.stop)] = col

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection="3d")

ax.scatter(theta_ratio[: len(emg)], delta_full[: len(emg)], emg, s=2, c=color.tolist())
ax.set_ylim(0, 0.2)

ax.set_xlabel("theta_ratio")
ax.set_ylabel("delta")
ax.set_zlabel("emg")